This is the data cleaning and feature engineering aspect of the Supervised Learning capstone.

In [1]:
#import the necessities
import numpy as np
import pandas as pd

In [2]:
#get the data which will become our model
df = pd.read_csv('D:/Downloads/cardsutf8.txt', sep='\t', encoding='latin1', low_memory=False)

In [3]:
#check the data (specifically the columns)
df.columns

Index(['index', 'id', 'artist', 'asciiName', 'availability', 'borderColor',
       'cardKingdomFoilId', 'cardKingdomId', 'colorIdentity', 'colorIndicator',
       'colors', 'convertedManaCost', 'duelDeck', 'edhrecRank',
       'faceConvertedManaCost', 'faceName', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isOnlineOnly', 'isOversized', 'isPromo', 'isReprint',
       'isReserved', 'isStarter', 'isStorySpotlight', 'isTextless',
       'isTimeshifted', 'keywords', 'layout', 'leadershipSkills', 'life',
       'loyalty', 'manaCost', 'mcmId', 'mcmMetaId', 'mtgArenaId',
       'mtgjsonV4Id', 'mtgoFoilId', 'mtgoId', 'multiverseId', 'name', 'number',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'purchaseUrls', 'rarity',
       'scryfallId', 'scryfallIllustrationId', 'scryfa

In [4]:
#strip away the useless information. for this model, we're focused
#on what the physical card has, not external attributes like price
#or cardkingdom/scryfall id. we're also not going to need things that
#can't affect power or toughness, like artist or frameEffects.
cards = df[['name', 'setCode', 'borderColor', 'colors', 'convertedManaCost', 'keywords', 
            'manaCost', 'power', 'rarity', 'supertypes', 'text', 'toughness', 'type']]
#probably drop originalReleaseDate since it has less than 2000 relative to the rest
#having 20,000. Sub-types probably can be dropped as well, since there's too many variances
#keywords has a group that commonly appears, so we'll use that as our baseline for that
#printings was going to be used as a way to say hoe many times an individual card has been
#printed, but it turns out that each printing is its own row, so that won't be necessary


In [5]:
#check the data to see what might be useful
cards.info()
#almost all of them are objects, but that's fine, as some of them
#just need a few columns cleaned and some of them can be changed
#into dummies
#for now, let's cull some of the more useless attributes.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56832 entries, 0 to 56831
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               56832 non-null  object 
 1   setCode            56832 non-null  object 
 2   borderColor        56832 non-null  object 
 3   colors             44292 non-null  object 
 4   convertedManaCost  56832 non-null  float64
 5   keywords           20648 non-null  object 
 6   manaCost           49509 non-null  object 
 7   power              26207 non-null  object 
 8   rarity             56832 non-null  object 
 9   supertypes         7840 non-null   object 
 10  text               55877 non-null  object 
 11  toughness          26207 non-null  object 
 12  type               56832 non-null  object 
dtypes: float64(1), object(12)
memory usage: 5.6+ MB


In [6]:
#gold and silver cards are not legal for play and borderless is just a frame type
#so strip away all but black and white border
cards = cards[(cards['borderColor'] != 'borderless') & (cards['borderColor'] != 'silver')
             & (cards['borderColor'] != 'gold')]

In [7]:
#we're only going to use this model for creatures, as those less variance than other
#types of cards
cards = cards[cards['type'].str.contains('Creature')]

In [8]:
#some of the powers aren't integers due to effects, so since those aren't easily
#predictable, we'll drop the problem cards
cards = cards[(cards['power'] != '*') & (cards['power'] != '1+*') & 
              (cards['power'] != '2+*') & (cards['power'] != '?')]
#same with toughness
cards = cards[(cards['toughness'] != '*') & (cards['toughness'] != '1+*')]
#and a weird one where the manaCost = {S}
cards = cards[cards['manaCost'] != '{S}']

In [9]:
#these columns are useful but would be more useful as numbers
cards['power'] = cards['power'].astype(int)
cards['toughness'] = cards['toughness'].astype(int)
cards['convertedManaCost'] = cards['convertedManaCost'].astype(int)

In [10]:
#the nulls in colors relate to colorless creatures, so we'll
#fill that with C to represent those. 
cards['colors'] = cards['colors'].fillna('C')
#the nulls in keywords means nothing there, so we'll fill
#that null with "No Keywords"
cards['keywords'] = cards['keywords'].fillna('No keywords')

In [11]:
#while I wish we could just use get_dummies to get what I want from here, there
#are too many different values and combination of values for get_dummies to be
#useful, so instead, we'll just use np.where to get the dummies individually
cards['White'] = np.where(cards['colors'].str.contains('W'), 1, 0)
cards['Blue'] = np.where(cards['colors'].str.contains('U'), 1, 0)
cards['Black'] = np.where(cards['colors'].str.contains('B'), 1, 0)
cards['Red'] = np.where(cards['colors'].str.contains('R'), 1, 0)
cards['Green'] = np.where(cards['colors'].str.contains('G'), 1, 0)

In [12]:
#same as above
cards['Deathtouch'] = np.where(cards['keywords'].str.contains('Deathtouch'), 1, 0)
cards['Defender'] = np.where(cards['keywords'].str.contains('Defender'), 1, 0)
cards['Double strike'] = np.where(cards['keywords'].str.contains('Double strike'), 1, 0)
cards['First strike'] = np.where(cards['keywords'].str.contains('First strike'), 1, 0)
cards['Flash'] = np.where(cards['keywords'].str.contains('Flash'), 1, 0)
cards['Flying'] = np.where(cards['keywords'].str.contains('Flying'), 1, 0)
cards['Haste'] = np.where(cards['keywords'].str.contains('Haste'), 1, 0)
cards['Hexproof1'] = np.where(cards['keywords'].str.contains('Hexproof'), 1, 0)
cards['Indestructible1'] = np.where(cards['keywords'].str.contains('Indestructible'), 1, 0)
cards['Lifelink'] = np.where(cards['keywords'].str.contains('Lifelink'), 1, 0)
cards['Menace1'] = np.where(cards['keywords'].str.contains('Menace'), 1, 0)
cards['Protection'] = np.where(cards['keywords'].str.contains('Protection'), 1, 0)
cards['Reach'] = np.where(cards['keywords'].str.contains('Reach'), 1, 0)
cards['Trample'] = np.where(cards['keywords'].str.contains('Trample'), 1, 0)
cards['Vigilance'] = np.where(cards['keywords'].str.contains('Vigilance'), 1, 0)
cards['Fear1'] = np.where(cards['keywords'].str.contains('Fear'), 1, 0)
cards['Shroud1'] = np.where(cards['keywords'].str.contains('Shroud'), 1, 0)
cards['Intimidate1'] = np.where(cards['keywords'].str.contains('Intimidate'), 1, 0)
cards['Regenerate1'] = np.where(cards['text'].str.contains('Regenerate'), 1, 0)
cards['Fight'] = np.where(cards['keywords'].str.contains('Fight'), 1, 0)
cards['Mill'] = np.where(cards['keywords'].str.contains('Mill'), 1, 0)
cards['Scry'] = np.where(cards['keywords'].str.contains('Scry'), 1, 0)

In [13]:
#Regenerate, Fear, Intimidate, and Shroud were all supplanted by keywords later,
#so to aid our analysis, we'll pretend that original and supplanted keywords, like indestructible and 
#regenerate are interchangable, which functionally, is a correct statement
cards['Regenerate / Indestructible'] = (cards['Indestructible1'] + cards['Regenerate1'])
cards['Fear / Intimidate / Menace'] = (cards['Intimidate1'] + cards['Menace1'] + cards['Fear1'])
cards['Shroud / Hexproof'] = cards['Hexproof1'] + cards['Shroud1']

In [14]:
#some of the cards are legendary, which might have an effect.
dummy = pd.get_dummies(cards['supertypes'])
cards['Legendary'] = dummy['Legendary'].astype(int)

In [15]:
#rarity has four possible values, so we'll use get the dummies
#for those and add them in
dummy = pd.get_dummies(cards['rarity'])
cards['Uncommon'] = dummy['uncommon'].astype(int)
cards['Rare'] = dummy['rare'].astype(int)
cards['Mythic'] = dummy['mythic'].astype(int)

In [16]:
#this is super messy but I don't know a better way to extract the number
#from inside the bracket. what we've done here is removed the brackets
#from each of them, but since str.replace('}', '') left values that 
#prevented me from being able to convert to int, so more precise and
#inelegant code was used here
##maybe do np.where to catch most of them (wouldn't get Phyrexian
##or 2/W)
cards['manaCost'] = cards['manaCost'].str.replace('W', '')
cards['manaCost'] = cards['manaCost'].str.replace('U', '')
cards['manaCost'] = cards['manaCost'].str.replace('B', '')
cards['manaCost'] = cards['manaCost'].str.replace('R', '')
cards['manaCost'] = cards['manaCost'].str.replace('G', '')
cards['manaCost'] = cards['manaCost'].str.replace('P', '')
cards['manaCost'] = cards['manaCost'].str.replace('C', '')
cards['manaCost'] = cards['manaCost'].str.replace('{/}', '{}')
cards['manaCost'] = cards['manaCost'].str.replace('{', '')
cards['manaCost'] = cards['manaCost'].str.replace('2/}', 'Z')
cards['manaCost'] = cards['manaCost'].str.replace('ZZZZZ', '10')
cards['manaCost'] = cards['manaCost'].str.replace('X}X}', '0}')
cards['manaCost'] = cards['manaCost'].str.replace('X}', '0}')
cards['manaCost'] = cards['manaCost'].str.replace('}}', '}')
cards['manaCost'] = cards['manaCost'].str.replace('}}', '}')
cards['manaCost'] = cards['manaCost'].str.replace('}}', '}')
cards['manaCost'] = cards['manaCost'].str.replace('}}', '}')
cards['manaCost'] = cards['manaCost'].str.replace('}}', '}')
cards['manaCost'] = cards['manaCost'].str.replace('0}2', '2')
cards['manaCost'] = cards['manaCost'].str.replace('1}', '1')
cards['manaCost'] = cards['manaCost'].str.replace('2}', '2')
cards['manaCost'] = cards['manaCost'].str.replace('3}', '3')
cards['manaCost'] = cards['manaCost'].str.replace('4}', '4')
cards['manaCost'] = cards['manaCost'].str.replace('5}', '5')
cards['manaCost'] = cards['manaCost'].str.replace('6}', '6')
cards['manaCost'] = cards['manaCost'].str.replace('7}', '7')
cards['manaCost'] = cards['manaCost'].str.replace('8}', '8')
cards['manaCost'] = cards['manaCost'].str.replace('9}', '9')
cards['manaCost'] = cards['manaCost'].str.replace('0}', '0')
cards['manaCost'] = cards['manaCost'].str.replace('}', '0')
cards['manaCost'] = cards['manaCost'].fillna(0)
cards['Generic Cost'] = cards['manaCost'].astype(int)

In [17]:
#specific cost signifies a requirement added while generic cost signifies
#the leftover amount (convertedManaCost - specific cost)
cards['Specific Cost'] = (cards['convertedManaCost'] - cards['Generic Cost']).astype(int)

In [18]:
#the keywords are given based on color, so since we have color defined, we can
#determine the likely keywords based on that. a single column that gives the number
#of keywords that the card has is more useful than knowing exactly what keyword
#the card has
cards['EG Keywords'] = (cards['Deathtouch'] + cards['Defender'] + cards['Double strike'] 
                        + cards['First strike'] + cards['Flash'] + cards['Flying'] + cards['Haste']
                        + cards['Lifelink'] + cards['Protection'] + cards['Reach'] + cards['Trample'] 
                        + cards['Vigilance'] + cards['Fight'] + cards['Mill'] + 
                        cards['Regenerate / Indestructible'] + cards['Fear / Intimidate / Menace']
                        + cards['Shroud / Hexproof'] + cards['Scry'] + cards['Fight'])

In [19]:
#this is a bit trickier. sets generally add new keywords that will appear only in
#that set, so we'll focus on this qualitatively rather than quantitatively.
#once my coding abilities become better, I may be able to return to this and
#find a better way to quantify this
cards['Set Keywords'] = cards['keywords']
old_evergreen = ['Deathtouch', 'Defender', 'Double strike', 'First strike', 'Fight',
                 'Flash', 'Flying', 'Haste', 'Lifelink', 'Protection', 'Scry',
                 'Reach', 'Trample', 'Vigilance', 'Fight', 'Mill', 'Regenerate',
                 'Indestructible', 'Fear', 'Intimidate', 'Menace', 'Shroud', 'Hexproof']
for keyword in old_evergreen:
    cards['Set Keywords'] = cards['Set Keywords'].str.replace(keyword,'-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace(',','')
cards['Set Keywords'] = cards['Set Keywords'].str.replace(' from','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('--','-')
cards['Set Keywords'] = cards['Set Keywords'].str.replace('-','No keywords')
cards['Set Keywords'] = np.where(cards['Set Keywords'] == 'No keywords', 0, 1)

In [20]:
#these are the columns that were useful for getting us other values but aren't
#useful or needed for the final analysis
cards = cards.drop(['Deathtouch', 'Defender', 'Double strike', 'First strike', 'Fight',
                 'Flash', 'Flying', 'Haste', 'Lifelink', 'Protection', 'Scry', 'Generic Cost',
                 'Reach', 'Trample', 'Vigilance', 'Fight', 'Mill', 'Regenerate / Indestructible',
                    'Fear / Intimidate / Menace', 'Shroud / Hexproof'], axis = 1)
cards = cards.drop(['borderColor', 'colors', 'keywords', 'manaCost', 'rarity', 
                    'supertypes', 'text', 'type'], axis = 1)
cards = cards.drop(['Indestructible1', 'Hexproof1', 'Menace1', 'Fear1', 'Shroud1',
                    'Intimidate1', 'Regenerate1'], axis = 1)

In [21]:
#I want to make the column names to look a little more uniform and move
#the potential targets to the end of the dataframe
dummy = pd.DataFrame()
dummy = cards.drop(['convertedManaCost', 'power', 'toughness', 'Legendary',
                    'Specific Cost', 'EG Keywords', 'Set Keywords'], axis = 1)
dummy['Converted Mana Cost'] = cards['convertedManaCost']
dummy['Specific Mana Cost'] = cards['Specific Cost']
dummy['Power'] = cards['power']
dummy['Toughness'] = cards['toughness']
dummy['Legendary'] = cards['Legendary']
dummy['EG Keywords'] = cards['EG Keywords']
dummy['Set Keywords'] = cards['Set Keywords']
cards = dummy.copy()

In [22]:
#as a baseline of comparison, I'd like to focus on the past two years or so of cards. to
#do that, I'll make a new df with only cards that appeared in sets in that timeframe
recent_sets = ['DOM', 'M19', 'GRN', 'RNA', 'WAR', 'M20', 'ELD', 'THB', 'IKO', 'M21', 'ZNR', 'KHM']
recent_cards = pd.DataFrame()
for set in recent_sets:
    recent_cards = recent_cards.append(cards[cards['setCode'] == set])

In [23]:
#while names and setCode are great for identification, they aren't necessary for
#the machine learning
cards = cards.drop(['name', 'setCode'], axis = 1)
cards = cards.reset_index()
cards = cards.drop(['index'], 1)
recent_cards = recent_cards.drop(['name', 'setCode'], axis = 1)
recent_cards = recent_cards.reset_index()
recent_cards = recent_cards.drop(['index'], 1)

In [24]:
#finally, we'll save these cleaned datasets and use them in modeling
cards.to_csv('D:/DSF Files/cards.txt', index=False, sep='\t')
recent_cards.to_csv('D:/DSF Files/recentcards.txt', index=False, sep='\t')